VEHICLE ROUTING WITH CAPACITY CONSTRAINTS

!/usr/bin/env python3
 Copyright 2010-2022 Google LLC
 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.


In [1]:
import requests
import pandas as pd
import requests
import time
import warnings
warnings.filterwarnings('ignore')

PART 1

creating the distance matrix

In [2]:
df = pd.read_csv("galveston_clusters.csv", comment="#")

subset = df.iloc[:25]

subset['x'] = subset['x'].round(2)
subset['y'] = subset['y'].round(2)

subset['x'] = subset['x'].astype(str)
subset['y'] = subset['y'].astype(str)


subset['coordinates_clean'] = subset['y'] + ',' + subset['x']

joined_string = ';'.join(subset['coordinates_clean'].astype(str))

joined_string

'-94.85,29.27;-94.82,29.27;-94.82,29.27;-94.86,29.29;-94.79,29.31;-94.8,29.28;-94.81,29.29;-94.79,29.31;-94.8,29.28;-94.8,29.28;-94.82,29.27;-94.77,29.3;-94.8,29.28;-94.81,29.28;-94.78,29.31;-94.79,29.3;-94.79,29.31;-94.78,29.31;-94.79,29.31;-94.81,29.28;-94.83,29.27;-94.79,29.31;-94.79,29.31;-94.78,29.31;-94.83,29.28'

In [11]:
def get_duration(a, b):
    starting_url = "https://api.mapbox.com/directions-matrix/v1/mapbox/driving/"
    url = starting_url + a + ';' + b
    params = {'access_token': 'pk.eyJ1Ijoic2FuZHJhbWF1cm8iLCJhIjoiY2xka3NuODFtMXFxeDN2bnUyaHUwNXllYiJ9.jz3sjWWEgPf8FSeCF2rQkQ'}
    r = requests.get(url, params=params)
    result = r.json()['durations'][0][1]
    return result


In [12]:
distance_matrix = []

for index, row in subset.iterrows():
    sub_matrix = []
    for index_2, row_2 in subset.iterrows():
        duration = get_duration(row['coordinates_clean'], row_2['coordinates_clean'])
        time.sleep(1)
        sub_matrix.append(duration)
        print(duration)
    distance_matrix.append(sub_matrix)

0
463
463
527.3
945.5
627.7
672.7
945.5
627.7
627.7
463
950.1
627.7
567.8
996.1
829.2
945.5
996.1
945.5
567.8
401.8
945.5
945.5
996.1
299.7
550.4
0
0
504.4
750.5
184.3
384.2
750.5
184.3
184.3
0
506.7
184.3
178.9
639.4
516.2
750.5
639.4
750.5
178.9
195.8
750.5
750.5
639.4
228.5
550.4
0
0
504.4
750.5
184.3
384.2
750.5
184.3
184.3
0
506.7
184.3
178.9
639.4
516.2
750.5
639.4
750.5
178.9
195.8
750.5
750.5
639.4
228.5
645.7
580.5
580.5
0
630.5
661.9
561.7
630.5
661.9
661.9
580.5
839.7
661.9
616.9
681.1
679.1
630.5
681.1
630.5
616.9
510.6
630.5
630.5
681.1
372
1053.5
771.5
771.5
623.2
0
554.8
577.9
0
554.8
554.8
771.5
381.8
554.8
716
225.7
318.5
0
225.7
0
716
925.2
0
0
225.7
738.1
627.5
216.7
216.7
658.9
566.2
0
332.2
566.2
0
0
216.7
322.4
0
174.3
455.1
331.9
566.2
455.1
566.2
174.3
412.5
566.2
566.2
455.1
371.4
647
401.1
401.1
508.4
604.4
261
0
604.4
261
261
401.1
478.7
261
217.2
655
377.4
604.4
655
604.4
217.2
518.7
604.4
604.4
655
365.9
1053.5
771.5
771.5
623.2
0
554.8
577.9
0
554.8
554.8


Part 2

Running Google OR Tools

In [15]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 3
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Duration'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')


if __name__ == '__main__':
    main()

Objective: 235523
Route for vehicle 0:
 0 ->  20 ->  10 ->  1 ->  2 ->  9 ->  8 ->  5 ->  6 ->  3 -> 0
Distance of the route: 2260m

Route for vehicle 1:
 0 ->  24 ->  15 ->  11 ->  12 ->  13 ->  19 -> 0
Distance of the route: 2175m

Route for vehicle 2:
 0 ->  23 ->  17 ->  14 ->  22 ->  21 ->  18 ->  16 ->  7 ->  4 -> 0
Distance of the route: 2288m

Maximum of the route distances: 2288m
